In [1]:
from trading import Environment
from trading.strategies.simulation import Sma2Porcentaje, Sma4Porcentaje
from trading.strategies.backtesting import Strategy2sma_porcentaje, Strategy4sma_porcentaje
from trading.adapter import Data2SmaPorcentaje, Data4SmaPorcentaje
from trading.utils import ACTIONS, BET_STATES
from datetime import datetime
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from backtesting import Backtest
import pandas as pd
import numpy as np

In [2]:
adapter = Data4SmaPorcentaje(n_sma1= 89, n_sma2= 8, n_sma3= 79, n_sma4= 45)
strategy = Sma4Porcentaje(factor_sll= 0.9254, facto_sls= 1.0137, factor_swl= 1.0317, factor_sws= 0.9426)

In [3]:
env = Environment('dataset.csv', data_adapter= adapter, cash= 1)

In [4]:
actual, ml_state = env.actual_state()
ml_state

open_time      1.577864e+12
open           7.229799e+03
high           7.231096e+03
low            7.229766e+03
close          7.230864e+03
sma1_prev     -9.911786e+00
sma1          -9.860667e+00
sma2          -2.926531e+00
sma2_prev     -3.565863e+00
reference      1.000000e+00
bet_state      2.000000e+00
price %        7.230864e+03
reference %    7.230864e+03
dtype: float64

In [12]:
env.get_summary()

Return [%]                                     99.0


In [13]:
env.get_money()

1

In [14]:
data = pd.read_csv("dataset.csv")
data = data.rename({'open':'Open',
                    'high':'High',
                    'low':'Low',
                    'close':'Close'},axis=1).drop(['volume'],axis=1)
data = data[data['open_time']%(5*60000)==0].copy()
data['open_time'] = pd.DatetimeIndex( data['open_time'].apply(lambda t:datetime.fromtimestamp(t/1000)) )
data = data.set_index('open_time')
data = data.sort_index(ascending=True)

In [15]:
bt = Backtest(data, Strategy4sma_porcentaje, cash= 1e6, commission= .0005, exclusive_orders=True)

In [16]:
bt.run(x = [89, 8, 79, 45, 1.0317, 0.9254, 0.9426, 1.0137])

Start                     2019-12-31 21:00:00
End                       2024-02-29 20:55:00
Duration                   1520 days 23:55:00
Exposure Time [%]                   87.857495
Equity Final [$]              36129810.139618
Equity Peak [$]               38157343.466844
Return [%]                        3512.981014
Buy & Hold Return [%]              753.257178
Return (Ann.) [%]                  136.374795
Volatility (Ann.) [%]              140.985954
Sharpe Ratio                         0.967293
Sortino Ratio                        3.768695
Calmar Ratio                         2.951576
Max. Drawdown [%]                  -46.204068
Avg. Drawdown [%]                   -1.264034
Max. Drawdown Duration      441 days 16:10:00
Avg. Drawdown Duration        1 days 12:53:00
# Trades                                  850
Win Rate [%]                        61.647059
Best Trade [%]                       9.240521
Worst Trade [%]                    -12.858639
Avg. Trade [%]                    

## Model

### Data

In [17]:
adapter = Data4SmaPorcentaje(n_sma1= 89, n_sma2= 8, n_sma3= 79, n_sma4= 45)
strategy = Sma4Porcentaje(factor_sll= 0.9254, facto_sls= 1.0137, factor_swl= 1.0317, factor_sws= 0.9426)

In [18]:
env = Environment('dataset.csv', data_adapter= adapter, cash= 1)

In [19]:
data, labels = [], []

state, ml_state = env.actual_state()
while True:
    action = strategy.next(state)

    # act = np.argmax(action)
    # if act == ACTIONS.OPEN_LONG.value or act == ACTIONS.OPEN_SHORT.value:
    #     temp_x, temp_y = [ml_state], [action]
    # elif (state.iloc[14] == BET_STATES.IN_LONG.value or state.iloc[14] == BET_STATES.IN_SHORT.value) \
    #       and act == ACTIONS.NONE.value:
    #     temp_x.append(ml_state)
    #     temp_y.append(action)
    # elif act == ACTIONS.CLOSE_POS.value:
    #     middle = (len(temp_x) // 2) - 1
        
    #     data.append(temp_x[0])
    #     data.append(temp_x[middle])
    data.append(ml_state)
        # labels.append(temp_y[0])
        # labels.append(temp_y[middle])
    labels.append(action)

    reward, state, ml_state, end = env.step(action)

    if end: break

In [ ]:
actions = np.zeros((4, ))
for label in labels:
    i = np.argmax(label)
    actions[i] += 1

In [ ]:
actions

array([524., 335., 849., 839.])

In [46]:
env.get_summary()

Return [%]                                   2582.8


In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

model = DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size= 0.5)
model = model.fit(np.array(X_train), np.array(y_train))

In [36]:
env.reset()
state, ml_state = env.actual_state()

while True:
    action = model.predict(np.array([ml_state]))[0]
    reward, state, ml_state, end = env.step(action)

    if end: break

In [37]:
env.get_summary()

Return [%]                                   1785.6


## Metricas

In [38]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score, recall_score, accuracy_score

In [39]:
results = model.predict(X_test)

In [32]:
c, d = [], []

for i in range(len(X_test)):
    if np.argmax(y_test[i]) == ACTIONS.CLOSE_POS.value:
        c.append(X_test[i])
        d.append(y_test[i])

In [ ]:
accuracy_score(d, model.predict(c))

0.6566265060240963

In [40]:
mae = mean_absolute_error(y_test, results)
mse = mean_squared_error(y_test, results)
f1 = f1_score(y_test, results, average= 'macro')
recall = recall_score(y_test, results, average= 'macro')
accuracy = accuracy_score(y_test, results)

print('Resultados: ')
print(f'MAE: {round(mae, 3):>30}')
print(f'MSE: {round(mse, 3):>30}')
print(f'Accuracy: {round(accuracy, 3):>25}')
print(f'F1 Score: {round(f1, 3):>25}')
print(f'Recall: {round(recall, 3):>27}')
print(f'Accuracy: {round(accuracy, 3):>25}')

Resultados: 
MAE:                            0.0
MSE:                            0.0
Accuracy:                       1.0
F1 Score:                     0.986
Recall:                       0.975
Accuracy:                       1.0
